In [1]:
import requests

In [2]:
import base64

In [3]:
import datetime

In [17]:
import json

In [4]:
from urllib.parse import urlencode

In [5]:
client_id = '880e1efab91840d1839bc9e887302e4c'
client_secret = 'f0ea5f81740b48e288d8e9ea15489d63'


In [7]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    token_url = "https://accounts.spotify.com/api/token"
    
    client_id = None
    client_secret = None
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        returns a base64 encoded string
        
        """
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_id == None or client_secret == None:
            raise Exception("You must set client_id and client_secret")
        
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
    "grant_type": "client_credentials"
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        
        r = requests.post(token_url, data = token_data, headers = token_headers)
        
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client")
            return False

        now = datetime.datetime.now()

        data = r.json()
        access_token = data["access_token"]
        expires_in = data["expires_in"] #seconds

        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):

        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token

    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}/tracks"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def search(self, query, search_type='artist' ): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()

In [8]:
spotify = SpotifyAPI(client_id, client_secret)

In [9]:
spotify.search("BTS", search_type = "artist")

{'artists': {'href': 'https://api.spotify.com/v1/search?query=BTS&type=artist&offset=0&limit=20',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX'},
    'followers': {'href': None, 'total': 18576005},
    'genres': ['k-pop', 'k-pop boy group'],
    'href': 'https://api.spotify.com/v1/artists/3Nrfpe0tUJi4K4DXYWgMUX',
    'id': '3Nrfpe0tUJi4K4DXYWgMUX',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/0c9057cb30520f9f883a220051260fc66a2f3ffa',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/34ee854082d0ea4d571e2ecf7f4aecea61270ad0',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/156147697b696d4a6180e037dc1e2a33117d8d4a',
      'width': 160}],
    'name': 'BTS',
    'popularity': 93,
    'type': 'artist',
    'uri': 'spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/51sg5jUqKu2tkbmPlwPrNH'},
    'fol

In [27]:
type(spotify.search("BTS", search_type = "artist"))

dict

In [35]:
bts_profile = spotify.search("BTS", search_type = "artist")["artists"]["items"][0]

In [36]:
bts_profile

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX'},
 'followers': {'href': None, 'total': 18576005},
 'genres': ['k-pop', 'k-pop boy group'],
 'href': 'https://api.spotify.com/v1/artists/3Nrfpe0tUJi4K4DXYWgMUX',
 'id': '3Nrfpe0tUJi4K4DXYWgMUX',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/0c9057cb30520f9f883a220051260fc66a2f3ffa',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/34ee854082d0ea4d571e2ecf7f4aecea61270ad0',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/156147697b696d4a6180e037dc1e2a33117d8d4a',
   'width': 160}],
 'name': 'BTS',
 'popularity': 93,
 'type': 'artist',
 'uri': 'spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX'}

In [37]:
bts_profile["followers"]["total"]

18576005

In [38]:
spotify.get_album("6mJZTV8lCqnwftYZa94bXS")

{'href': 'https://api.spotify.com/v1/albums/6mJZTV8lCqnwftYZa94bXS/tracks?offset=0&limit=20',
 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Nrfpe0tUJi4K4DXYWgMUX'},
     'href': 'https://api.spotify.com/v1/artists/3Nrfpe0tUJi4K4DXYWgMUX',
     'id': '3Nrfpe0tUJi4K4DXYWgMUX',
     'name': 'BTS',
     'type': 'artist',
     'uri': 'spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX'}],
   'available_markets': ['AD',
    'AE',
    'AR',
    'AT',
    'AU',
    'BE',
    'BG',
    'BH',
    'BO',
    'BR',
    'CA',
    'CH',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DE',
    'DK',
    'DO',
    'DZ',
    'EC',
    'EE',
    'EG',
    'ES',
    'FI',
    'FR',
    'GB',
    'GR',
    'GT',
    'HN',
    'HU',
    'ID',
    'IE',
    'IL',
    'IN',
    'IS',
    'IT',
    'JO',
    'KW',
    'LB',
    'LI',
    'LT',
    'LU',
    'LV',
    'MA',
    'MC',
    'MT',
    'MX',
    'MY',
    'NI',
    'NL',
    'NO',
    'NZ',
    'OM',
    'PA',
   